In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns
import pydicom
import time
import gc
import operator 
from apex import amp 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
from apex import amp
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
SEED = 8153
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device = "cpu"
torch.cuda.set_device(device)
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet169-folds0 results')

In [3]:
def get_submission(test_df,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [4]:
old_train_df = pd.read_csv(data_dir+'train.csv')
old_train_df.shape
old_train_df=old_train_df[~old_train_df.PatientID.isin(bad_images)].reset_index(drop=True)
old_train_df=old_train_df.drop_duplicates().reset_index(drop=True)
old_train_df.shape
old_train_df.head()

train_df = pd.read_csv(data_dir+'train_stage2.csv')
train_df.shape


old_test_df = pd.read_csv(data_dir+'test.csv')
old_test_df.head()
assert old_test_df.shape[0]+old_train_df.shape[0]==train_df.shape[0]

n_splits=3
old_split_sid = old_train_df.PID.unique()
old_split_sid.shape[0]
old_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(old_split_sid))


new_split_sid=np.array(list(set(train_df.PID.unique()).difference(old_train_df.PID.unique())))
new_split_sid.shape[0]
new_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(new_split_sid))
split_sid=np.concatenate([old_split_sid,new_split_sid])
split_sid.shape[0]

splits=[]
for i in range(len(old_splits)):
    splits.append((np.concatenate([old_splits[i][0],new_splits[i][0]+old_split_sid.shape[0]]),
                  np.concatenate([old_splits[i][1],new_splits[i][1]+old_split_sid.shape[0]])))

for i in range(n_splits):
    for j in range(n_splits):
        idx_train = train_df[train_df.PID.isin(set(split_sid[splits[i][0]]))].index.values
        idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[j][1]]))].index.values
        if i==j:
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]+idx_validate.shape[0]
        else:
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]

(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


(752797, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


17079

1859

18938

In [5]:
test_df = pd.read_csv(data_dir+'test_stage2.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,0fbf6a978,0.5,0.5,0.5,0.5,0.5,0.5,48eaa460c6,3556ae0e,3710aff1c5,40,100,37.690857,-125.0,-117.830185
1,d62ec3412,0.5,0.5,0.5,0.5,0.5,0.5,389cec2e11,c667d610,8e38d86ede,"['00036', '00036']","['00080', '00080']",762.200043,-66.0,-10.000000
2,cb544194b,0.5,0.5,0.5,0.5,0.5,0.5,a6ebe3ca4f,9389d3b6,7040866911,"['00036', '00036']","['00080', '00080']",818.400024,-105.0,6.000000
3,0d62513ec,0.5,0.5,0.5,0.5,0.5,0.5,01c388cfe3,79cdc6e4,07c298b5be,30,80,31.857132,-125.0,-112.984024
4,fc45b2151,0.5,0.5,0.5,0.5,0.5,0.5,d16ad143cb,39eb6eb6,6a15b35eae,"['00036', '00036']","['00080', '00080']",424.099976,-102.5,14.500000


In [6]:
def my_loss(y_pred,y_true,weights):
    if len(y_pred.shape)==len(y_true.shape):
        loss = F.binary_cross_entropy_with_logits(y_pred,y_true,weights.repeat(y_pred.shape[0],1))
    else:
        loss0 = F.binary_cross_entropy_with_logits(y_pred,y_true[...,0],weights.repeat(y_pred.shape[0],1),reduction='none')
        loss1 = F.binary_cross_entropy_with_logits(y_pred,y_true[...,1],weights.repeat(y_pred.shape[0],1),reduction='none')
        loss = (y_true[...,2]*loss0+(1.0-y_true[...,2])*loss1).mean() 
    return loss

In [7]:
class parameter_scheduler():
    def __init__(self,model,do_first=['classifier'],num_epoch=1):
        self.model=model
        self.do_first = do_first
        self.num_epoch=num_epoch
    def __call__(self,epoch):
        if epoch>=self.num_epoch:
            for n,p in self.model.named_parameters():
                p.requires_grad=True
        else:
            for n,p in self.model.named_parameters():
                p.requires_grad= any(nd in n for nd in self.do_first)


In [8]:
def get_optimizer_parameters(model,klr):
    zero_layer=['conv0','norm0','ws_norm']
    param_optimizer = list(model.named_parameters())
    num_blocks=4
    no_decay=['bias']
    optimizer_grouped_parameters=[
        {'params': [p for n, p in param_optimizer if (not any(nd in n for nd in no_decay) and any(nd in n for nd in zero_layer))], 'lr':klr*2e-5,'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)  and any(nd in n for nd in zero_layer)], 'lr':klr*2e-5, 'weight_decay': 0.0}
        ]
    optimizer_grouped_parameters.extend([
        {'params': [p for n, p in param_optimizer if (not any(nd in n for nd in no_decay) and ('wso' in n))], 'lr':klr*1e-5,'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)  and ('wso' in n)], 'lr':klr*1e-5, 'weight_decay': 0.0}
        ])
    optimizer_grouped_parameters.extend([
        {'params': [p for n, p in param_optimizer if (not any(nd in n for nd in no_decay) and ('classifier' in n))], 'lr':klr*1e-3,'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)  and ('classifier' in n)], 'lr':klr*1e-3, 'weight_decay': 0.0}
        ])
    for i in range(num_blocks):
        optimizer_grouped_parameters.extend([
        {'params': [p for n, p in param_optimizer if (not any(nd in n for nd in no_decay) and ('denseblock{}'.format(i+1) in n))], 'lr':klr*(2.0**i)*2e-5,'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)  and ('denseblock{}'.format(i+1) in n)], 'lr':klr*(2.0**i)*2e-5, 'weight_decay': 0.0}
        ])
    optimizer_grouped_parameters.extend([
        {'params': [p for n, p in param_optimizer if (not any(nd in n for nd in no_decay) and ('norm5' in n))], 'lr':klr*1e-4,'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)  and ('norm5' in n)], 'lr':klr*1e-4, 'weight_decay': 0.0}
        ])
    return(optimizer_grouped_parameters)

    

In [9]:
for num_split in range(3):
    model_name,version =  'Densenet169_3' , 'classifier_splits'
    print (model_name,version,num_split)
    model = MyDenseNet(models.densenet169(pretrained=True),
                       len(hemorrhage_types),
                       num_channels=3,
                       drop_out=0.2,
                       wso=((40,80),(80,200),(600,2800)),
                       strategy='none',
                       dont_do_grad=[],
                       pool_type='max',
                       extra_pool=8,
                       return_features=True
                       )
    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    _=model.to(device)
    transform=MyTransform(mean_change=15,
                          std_change=0,
                          flip=True,
                          zoom=(0.2,0.2),
                          rotate=30,
                          out_size=512,
                          shift=10,
                          normal=False)
    transform_val=MyTransform(out_size=512)
    indexes=np.arange(test_df.shape[0]).repeat(8)
    imagedataset_test=D.Subset(ImageDataset(test_df,transform=transform.random,base_path=test_images_dir,
                                  window_eq=False,equalize=False,rescale=True),indexes)
    pred,features = model_run(model,imagedataset_test,do_apex=True,batch_size=96,num_workers=18)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test_stage2',num_split),'wb')
    pickle.dump(features,pickle_file,protocol=4)
    pickle_file.close()
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'predictions_test_stage2',num_split),'wb')
    pickle.dump(pred,pickle_file,protocol=4)
    pickle_file.close()

Densenet169_3 classifier_splits 0


<All keys matched successfully>


Densenet169_3 classifier_splits 1


<All keys matched successfully>

In [12]:
for num_split in range(3):
    model_name,version =  'Densenet161_3' , 'classifier_splits'
    print (model_name,version,num_split)
    model = MyDenseNet(models.densenet161(pretrained=True),
                       len(hemorrhage_types),
                       num_channels=3,
                       drop_out=0.2,
                       wso=((40,80),(80,200),(600,2800)),
                       strategy='none',
                       dont_do_grad=[],
                       pool_type='max',
                       extra_pool=4,
                       return_features=True
                       )
    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    _=model.to(device)
    transform=MyTransform(flip=True,zoom=0.05,rotate=15,out_size=512,shift=40)
    transform_val=MyTransform(out_size=512)
    indexes=np.arange(test_df.shape[0]).repeat(8)
    imagedataset_test=D.Subset(ImageDataset(test_df,transform=transform.random,base_path=test_images_dir,
                                  window_eq=False,equalize=False,rescale=True),indexes)
    pred,features = model_run(model,imagedataset_test,do_apex=True,batch_size=96,num_workers=18)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test_stage2',num_split),'wb')
    pickle.dump(features,pickle_file,protocol=4)
    pickle_file.close()
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'predictions_test_stage2',num_split),'wb')
    pickle.dump(pred,pickle_file,protocol=4)
    pickle_file.close()

Densenet161_3 classifier_splits 0


<All keys matched successfully>


Densenet161_3 classifier_splits 1


<All keys matched successfully>


Densenet161_3 classifier_splits 2


<All keys matched successfully>

In [9]:
for num_split in [2]: #range(3):
    model_name,version =  'se_resnext101_32x4d' , 'classifier_splits'
    idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].index.values
    model = MySENet(pretrainedmodels.__dict__['se_resnext101_32x4d'](num_classes=1000, pretrained='imagenet'),
                       len(hemorrhage_types),
                       num_channels=3,
                       dropout=0.2,
                       wso=((40,80),(80,200),(40,400)),
                       dont_do_grad=[],
                       extra_pool=8,
                       return_features=True
                       )
    wait_for_weights = True
    while wait_for_weights:
        try:
            model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
            wait_for_weights=False
        except OSError as e:
            time.sleep(10)
    _=model.to(device)
    print (models_format.format(model_name,version,num_split))
    transform=MyTransform(mean_change=15,
                          std_change=0,
                          flip=True,
                          zoom=(0.2,0.2),
                          rotate=30,
                          out_size=512,
                          shift=10,
                          normal=False)
    transform_val=MyTransform(out_size=512)
    indexes=np.arange(train_df.shape[0]).repeat(4)
    train_dataset=D.Subset(ImageDataset(train_df,transform=transform.random,base_path=train_images_dir,
                              window_eq=False,equalize=False,rescale=True),indexes)
#    pred,features = model_run(model,train_dataset,do_apex=True,batch_size=96,num_workers=14)

#    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'wb')
#    pickle.dump(features,pickle_file,protocol=4)
#    pickle_file.close()

#    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'predictions_train_tta',num_split),'wb')
#    pickle.dump(pred,pickle_file,protocol=4)
#    pickle_file.close()


#    my_loss(pred[(idx_validate*4+np.arange(4)[:,None]).transpose(1,0)].mean(1),
#            torch.tensor(train_df[hemorrhage_types].values[idx_validate],dtype=torch.float),
#            torch.tensor([1.,1.,1.,1.,1.,2.]))
    
    indexes=np.arange(test_df.shape[0]).repeat(8)
    imagedataset_test=D.Subset(ImageDataset(test_df,transform=transform.random,base_path=test_images_dir,
                                  window_eq=False,equalize=False,rescale=True),indexes)
    pred,features = model_run(model,imagedataset_test,do_apex=True,batch_size=96,num_workers=12)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test_stage2',num_split),'wb')
    pickle.dump(features,pickle_file,protocol=4)
    pickle_file.close()
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'predictions_test_stage2',num_split),'wb')
    pickle.dump(pred,pickle_file,protocol=4)
    pickle_file.close()

<All keys matched successfully>

model_se_resnext101_32x4d_version_classifier_splits_split_2.pth
